In [25]:
from gurobipy import *
import numpy as np

# Model 1: Non-looping buses

In [26]:
bus_cap = 50
t_1 = 10 # time taken to travel from stop 1 to 2
t_2 = 10 # time taken to travel from stop 2 to 3

w_1 = 10 # waiting time for next bus at stop 1
w_2 = 10 # waiting time for next bus at stop 2

arcs_waiting, waiting_time, capacity_waiting = multidict({
    ("1", "4") : [w_1, 9999],
    ("2", "5") : [w_2, 9999],
    ("4", "7") : [w_1, 9999],
    ("5", "8") : [w_2, 9999],
    ("7", "10") : [w_1, 9999],
    ("8", "11") : [w_2, 9999],
    ("10", "13") : [w_1, 9999],
    ("11", "14") : [w_2, 9999],
    ("13", "16") : [w_1, 9999],
    ("14", "17") : [w_2, 9999],
    ("16", "19") : [999, 9999],
    ("17", "20") : [999, 9999]
})

arcs_travelling, travelling_time, capacity_travelling = multidict({
    ("1", "2") : [t_1, bus_cap],
    ("2", "3") : [t_2, bus_cap],
    ("4", "5") : [t_1, bus_cap],
    ("5", "6") : [t_2, bus_cap],
    ("7", "8") : [t_1, bus_cap],
    ("8", "9") : [t_2, bus_cap],
    ("10", "11") : [t_1, bus_cap],
    ("11", "12") : [t_2, bus_cap],
    ("13", "14") : [t_1, bus_cap],
    ("14", "15") : [t_2, bus_cap],
    ("16", "17") : [t_1, bus_cap],
    ("17", "18") : [t_2, bus_cap] # Last nodes should have infinite capacity
})

node_inflow, inflow = multidict({
    "1" : [140],
    "2" : [70],
    "4" : [140],
    "5" : [70],
    "7" : [140],
    "8" : [70],
    "10" : [140],
    "11" : [70],
    "13" : [140],
    "14" : [70],
    "16" : [140],
    "17" : [70]
})

node_outflow = ["2","3","5","6","8","9","11","12","14","15","17","18"]

In [27]:
# Calculate sum of total inflows
sum([x[1] for x in inflow.items()])

1260

In [28]:
m1 = Model("Model")

waiting_arcs = m1.addVars(arcs_waiting, lb = [0,]*12, name = 'Arcs Waiting') 
travelling_arcs = m1.addVars(arcs_travelling, lb = [0,]*12, name = 'Arcs travelling') 
outflow = m1.addVars(node_outflow, name = "outflow")

m1.setObjective(quicksum(waiting_arcs[i]*waiting_time[i] for i in arcs_waiting), GRB.MINIMIZE)

# Flow conservation
m1.addConstr(inflow["1"] == travelling_arcs[("1", "2")] + waiting_arcs[("1", "4")], name = 'Node 1 Conservation')
m1.addConstr(inflow["2"]  + travelling_arcs[("1", "2")] == travelling_arcs[("2", "3")] + waiting_arcs[("2", "5")] + outflow["2"], name = 'Node 2 Conservation')
m1.addConstr(travelling_arcs[("2", "3")] == outflow["3"], name = 'Node 3 Conservation')
m1.addConstr(inflow["4"] + waiting_arcs[("1", "4")] == travelling_arcs[("4", "5")] + waiting_arcs[("4", "7")], name = 'Node 4 Conservation')
m1.addConstr(inflow["5"] + travelling_arcs[("4", "5")] + waiting_arcs[("2", "5")] == travelling_arcs[("5", "6")] + outflow["5"] + waiting_arcs[("5", "8")], name = 'Node 5 Conservation')
m1.addConstr(travelling_arcs[("5", "6")] == outflow["6"], name = 'Node 6 Conservation')
m1.addConstr(inflow["7"] + waiting_arcs[("4", "7")] == travelling_arcs[("7", "8")] + waiting_arcs[("7", "10")], name = 'Node 7 Conservation')
m1.addConstr(inflow["8"] + travelling_arcs[("7", "8")] + waiting_arcs[("5", "8")] == travelling_arcs[("8", "9")] + outflow["8"] + waiting_arcs[("8", "11")], name = 'Node 8 Conservation')
m1.addConstr(travelling_arcs[("8", "9")] == outflow["9"], name = 'Node 9 Conservation')
m1.addConstr(inflow["10"] + waiting_arcs[("7", "10")] == travelling_arcs[("10", "11")] + waiting_arcs[("10", "13")], name = 'Node 10 Conservation')
m1.addConstr(inflow["11"] + travelling_arcs[("10", "11")] + waiting_arcs[("8", "11")] == travelling_arcs[("11", "12")] + outflow["11"] + waiting_arcs[("11", "14")], name = 'Node 11 Conservation')
m1.addConstr(travelling_arcs[("11", "12")] == outflow["12"], name = 'Node 12 Conservation')
m1.addConstr(inflow["13"] + waiting_arcs[("10", "13")] == travelling_arcs[("13", "14")] + waiting_arcs[("13", "16")], name = 'Node 13 Conservation')
m1.addConstr(inflow["14"] + travelling_arcs[("13", "14")] + waiting_arcs[("11", "14")] == travelling_arcs[("14", "15")] + outflow["14"] + waiting_arcs[("14", "17")], name = 'Node 14 Conservation')
m1.addConstr(travelling_arcs[("14", "15")] == outflow["15"], name = 'Node 15 Conservation')
m1.addConstr(inflow["16"]  + waiting_arcs[("13", "16")] == travelling_arcs[("16", "17")] + waiting_arcs[("16","19")], name = 'Node 16 Conservation')
m1.addConstr(inflow["17"] + travelling_arcs[("16", "17")] + waiting_arcs[("14", "17")] == travelling_arcs[("17", "18")]+ outflow["17"] + waiting_arcs[("17","20")], name = 'Node 17 Conservation')
m1.addConstr(travelling_arcs[("17", "18")] == outflow["18"], name = 'Node 18 Conservation')

# Flow conservation (vectorized)
#node_len = len(node)
#for i in range(1+3, node_len + 1 - 3, 3): # 4, 7, 10, 13
#    m1.addConstr(inflow[str(i)] + waiting_arcs[(str(i-3), str(i))] == travelling_arcs[(str(i)), str(i+1)] + waiting_arcs[(str(i), str(i+3))], name = "Node " + str(i) + " Conservation")
#for i in range(2+3, node_len  + 1 - 3, 3): # 5, 8, 11, 14
#    m1.addConstr(inflow[str(i)] + waiting_arcs[(str(i-3), str(i))] + travelling_arcs[(str(i-1), str(i))] == travelling_arcs[(str(i)), str(i+1)] + waiting_arcs[(str(i), str(i+3))] + outflow[str(i)], name = "Node " + str(i) + " Conservation")
#for i in range(3, node_len  + 1, 3): # 3, 6, 9, 12, 15, 18
#    m1.addConstr(travelling_arcs[str(i-1), str(i)] == outflow[str(i)], name = "Node " + str(i) + " Conservation")
#m1.addConstr(inflow["1"] == travelling_arcs[("1", "2")] + waiting_arcs[("1", "4")], name = 'Node 1 Conservation')
#m1.addConstr(inflow["2"]  + travelling_arcs[("1", "2")] == travelling_arcs[("2", "3")] + waiting_arcs[("2", "5")] + outflow["2"], name = 'Node 2 Conservation')
#m1.addConstr(inflow["16"]  + waiting_arcs[("13", "16")] == travelling_arcs[("16", "17")], name = 'Node 16 Conservation')
#m1.addConstr(inflow["17"] + travelling_arcs[("16", "17")] + waiting_arcs[("14", "17")] == travelling_arcs[("17", "18")]+ outflow["17"], name = 'Node 17 Conservation')
    
#outflow <= no. of travelling passengers
for i in outflow:
    m1.addConstr(outflow[i] <= travelling_arcs[(str(int(i)-1),i)])

#Bus Capacity
for i in arcs_travelling:
    m1.addConstr(bus_cap >= travelling_arcs[i], name = 'Bus Capacity' + str(i))

m1.optimize()
        
print("Primal Solutions")
for v in m1.getVars():
    print("%s %g" % (v.varName, v.x))


Optimize a model with 42 rows, 36 columns and 94 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+02]
Presolve removed 42 rows and 36 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7584000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  6.758400000e+05
Primal Solutions
Arcs Waiting[1,4] 90
Arcs Waiting[2,5] 20
Arcs Waiting[4,7] 180
Arcs Waiting[5,8] 40
Arcs Waiting[7,10] 270
Arcs Waiting[8,11] 60
Arcs Waiting[10,13] 360
Arcs Waiting[11,14] 80
Arcs Waiting[13,16] 450
Arcs Waiting[14,17] 100
Arcs Waiting[16,19] 540
Arcs Waiting[17,20] 120
Arcs travelling[1,2] 50
Arcs travelling[2,3] 50
Arcs travelling[4,5] 50
Arcs travelling[5,6] 50
Arcs travelling[7,8] 50
Arcs travelling[8,9] 50
Arcs travelling[10,11] 50
Arcs travelling[11

In [29]:
# Print objective value
print('Obj: %g' % m1.objVal)

Obj: 675840


In [30]:
# Print shadow prices and sensitivity ranges
m1.printAttr(['Pi','SARHSLow','SARHSUp'])


  Constraint           Pi     SARHSLow      SARHSUp 
---------------------------------------------------
Node 1 Conservation         1049           50       1e+100 
Node 2 Conservation        -1049      -1e+100          -50 
Node 3 Conservation            0            0           50 
Node 4 Conservation        -1039      -1e+100           40 
Node 5 Conservation        -1039      -1e+100          -30 
Node 6 Conservation            0            0           50 
Node 7 Conservation        -1029      -1e+100          130 
Node 8 Conservation        -1029      -1e+100          -10 
Node 9 Conservation            0            0           50 
Node 10 Conservation        -1019      -1e+100          220 
Node 11 Conservation        -1019      -1e+100           10 
Node 12 Conservation            0            0           50 
Node 13 Conservation        -1009      -1e+100          310 
Node 14 Conservation        -1009      -1e+100           30 
Node 15 Conservation            0            0   

# Model 2: Looping Buses

In [31]:
bus_cap = 50
t_1 = 10 # time taken to travel from stop 1 to 2
t_2 = 10 # time taken to travel from stop 2 to 3

w_1 = 10 # waiting time for next bus at stop 1
w_2 = 10 # waiting time for next bus at stop 2
w_3 = 10 # waiting time for next bus at stop 3

d_1 = 20 # time taken to from stop 3 to 1 (diagonal)

arcs_waiting, waiting_time, capacity_waiting = multidict({
    ("1", "4") : [w_1, 99999],
    ("2", "5") : [w_2, 99999],
    ("3", "6") : [w_3, 99999],
    ("4", "7") : [w_1, 99999],
    ("5", "8") : [w_2, 99999],
    ("6", "9") : [w_3, 99999],
    ("7", "10") : [w_1, 99999],
    ("8", "11") : [w_2, 99999],
    ("9", "12") : [w_3, 99999],
    ("10", "13") : [w_1, 99999],
    ("11", "14") : [w_2, 99999],
    ("12", "15") : [w_3, 99999],
    ("13", "16") : [w_1, 99999],
    ("14", "17") : [w_2, 99999],
    ("16", "19") : [999, 99999],
    ("17", "20") : [999, 99999]
})

arcs_travelling, travelling_time, capacity_travelling = multidict({
    ("1", "2") : [t_1, bus_cap],
    ("2", "3") : [t_2, bus_cap],
    ("4", "5") : [t_1, bus_cap],
    ("5", "6") : [t_2, bus_cap],
    ("7", "8") : [t_1, bus_cap],
    ("8", "9") : [t_2, bus_cap],
    ("10", "11") : [t_1, bus_cap],
    ("11", "12") : [t_2, bus_cap],
    ("13", "14") : [t_1, bus_cap],
    ("14", "15") : [t_2, bus_cap],
    ("16", "17") : [t_1, bus_cap],
    ("17", "18") : [t_2, bus_cap], # Last nodes should have infinite capacity
    ("3", "4") : [d_1, bus_cap],
    ("6", "7") : [d_1, bus_cap],
    ("9", "10") : [d_1, bus_cap],
    ("12", "13") : [d_1, bus_cap],
    ("15", "16") : [d_1, bus_cap]
})

node_inflow, inflow = multidict({
    "1" : [140],
    "2" : [35],
    "3" : [35],
    "4" : [140],
    "5" : [35],
    "6" : [35],
    "7" : [140],
    "8" : [35],
    "9" : [35],
    "10" : [140],
    "11" : [35],
    "12" : [35],
    "13" : [140],
    "14" : [35],
    "15" : [35],
    "16" : [140],
    "17" : [70]
})

node_outflow = ["2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18"]

node = [str(x) for x in list(range(1,19))]

In [32]:
# Calculate sum of total inflows
sum([x[1] for x in inflow.items()])

1260

In [33]:
m2 = Model("Model")

waiting_arcs = m2.addVars(arcs_waiting, lb = [0,]*len(arcs_waiting), name = 'Arcs Waiting') 
travelling_arcs = m2.addVars(arcs_travelling, lb = [0,]*len(arcs_travelling), name = 'Arcs travelling')  
outflow = m2.addVars(node_outflow, name = "outflow")

m2.setObjective(quicksum(waiting_arcs[i]*waiting_time[i] for i in arcs_waiting), GRB.MINIMIZE)

# Flow conservation
m2.addConstr(inflow["1"] == travelling_arcs[("1", "2")] + waiting_arcs[("1", "4")], name = 'Node 1 Conservation')
m2.addConstr(inflow["2"]  + travelling_arcs[("1", "2")] == travelling_arcs[("2", "3")] + waiting_arcs[("2", "5")] + outflow["2"], name = 'Node 2 Conservation')
m2.addConstr(travelling_arcs[("2", "3")] + inflow[("3")] == outflow["3"] + waiting_arcs[("3","6")] + travelling_arcs[("3","4")], name = 'Node 3 Conservation')
m2.addConstr(inflow["4"] + waiting_arcs[("1", "4")] + travelling_arcs[("3","4")] == travelling_arcs[("4", "5")] + waiting_arcs[("4", "7")] + outflow[("4")], name = 'Node 4 Conservation')
m2.addConstr(inflow["5"] + travelling_arcs[("4", "5")] + waiting_arcs[("2", "5")] == travelling_arcs[("5", "6")] + outflow["5"] + waiting_arcs[("5", "8")], name = 'Node 5 Conservation')
m2.addConstr(travelling_arcs[("5", "6")] + inflow["6"] + waiting_arcs[("3", "6")] == outflow["6"] + waiting_arcs[("6","9")] + travelling_arcs[("6","7")], name = 'Node 6 Conservation')
m2.addConstr(inflow["7"] + waiting_arcs[("4", "7")] + travelling_arcs[("6", "7")] == travelling_arcs[("7", "8")] + waiting_arcs[("7", "10")] + outflow["7"], name = 'Node 7 Conservation')
m2.addConstr(inflow["8"] + travelling_arcs[("7", "8")] + waiting_arcs[("5", "8")] == travelling_arcs[("8", "9")] + outflow["8"] + waiting_arcs[("8", "11")], name = 'Node 8 Conservation')
m2.addConstr(travelling_arcs[("8", "9")] + inflow["9"] + waiting_arcs[("6","9")] == outflow["9"] + waiting_arcs[("9","12")] + travelling_arcs[("9","10")], name = 'Node 9 Conservation')
m2.addConstr(inflow["10"] + waiting_arcs[("7", "10")] + travelling_arcs[("9","10")] == travelling_arcs[("10", "11")] + waiting_arcs[("10", "13")] + outflow["10"], name = 'Node 10 Conservation')
m2.addConstr(inflow["11"] + travelling_arcs[("10", "11")] + waiting_arcs[("8", "11")] == travelling_arcs[("11", "12")] + outflow["11"] + waiting_arcs[("11", "14")], name = 'Node 11 Conservation')
m2.addConstr(travelling_arcs[("11", "12")] + inflow["12"] + waiting_arcs[("9","12")] == outflow["12"] + waiting_arcs[("12","15")] + travelling_arcs[("12","13")] , name = 'Node 12 Conservation')
m2.addConstr(inflow["13"] + waiting_arcs[("10", "13")] + travelling_arcs[("12","13")] == travelling_arcs[("13", "14")] + waiting_arcs[("13", "16")] + outflow["13"], name = 'Node 13 Conservation')
m2.addConstr(inflow["14"] + travelling_arcs[("13", "14")] + waiting_arcs[("11", "14")] == travelling_arcs[("14", "15")] + outflow["14"] + waiting_arcs[("14", "17")], name = 'Node 14 Conservation')
m2.addConstr(travelling_arcs[("14", "15")] +inflow["15"] + waiting_arcs[("12", "15")] == outflow["15"] + travelling_arcs[("15","16")], name = 'Node 15 Conservation')
m2.addConstr(inflow["16"]  + waiting_arcs[("13", "16")] + travelling_arcs[("15", "16")] == travelling_arcs[("16", "17")] + waiting_arcs[("16","19")] + outflow["16"], name = 'Node 16 Conservation')
m2.addConstr(inflow["17"] + travelling_arcs[("16", "17")] + waiting_arcs[("14", "17")] == travelling_arcs[("17", "18")]+ outflow["17"] + waiting_arcs[("17","20")], name = 'Node 17 Conservation')
m2.addConstr(travelling_arcs[("17", "18")] == outflow["18"], name = 'Node 18 Conservation')

#Flow conservation (vectorized)
#node_len = len(node)
#for i in range(1+3, node_len + 1 - 3, 3): # 4, 7, 10, 13
#    m2.addConstr(inflow[str(i)] + waiting_arcs[(str(i-3), str(i))] + diagonal_arcs[(str(i-1), str(i))] == travelling_arcs[(str(i)), str(i+1)] + waiting_arcs[(str(i), str(i+3))] + outflow[str(i)], name = "Node " + str(i) + " Conservation")
#for i in range(2+3, node_len  + 1 - 3, 3): # 5, 8, 11, 14
#    m2.addConstr(inflow[str(i)] + waiting_arcs[(str(i-3), str(i))] + travelling_arcs[(str(i-1), str(i))] == travelling_arcs[(str(i)), str(i+1)] + waiting_arcs[(str(i), str(i+3))] + outflow[str(i)], name = "Node " + str(i) + " Conservation")
#for i in range(6, node_len, 3): # 3, 6, 9, 12, 15
#    m2.addConstr(travelling_arcs[str(i-1), str(i)] == outflow[str(i)] + diagonal_arcs[(str(i), str(i+1))], name = "Node " + str(i) + " Conservation")
#m2.addConstr(inflow["1"] == travelling_arcs[("1", "2")] + waiting_arcs[("1", "4")], name = 'Node 1 Conservation')
#m2.addConstr(inflow["2"]  + travelling_arcs[("1", "2")] == travelling_arcs[("2", "3")] + waiting_arcs[("2", "5")] + outflow["2"], name = 'Node 2 Conservation')
#m2.addConstr(inflow["16"]  + waiting_arcs[("13", "16")] + diagonal_arcs[("15", "16")] == travelling_arcs[("16", "17")] + waiting_arcs[("16","19")], name = 'Node 16 Conservation')
#m2.addConstr(inflow["17"] + travelling_arcs[("16", "17")] + waiting_arcs[("14", "17")] == travelling_arcs[("17", "18")]+ outflow["17"] + waiting_arcs[("17","20")], name = 'Node 17 Conservation')
#m2.addConstr(travelling_arcs[("17", "18")] == outflow["18"], name = 'Node 18 Conservation')
    
#Demand
#for i in arcs_travelling:
#    try:
#        m2.addConstr(outflow[i[1]] <= travelling_arcs[i], name = 'Demand' + str(i))
#    except GurobiError:
#        continue
    
#Demand 2
for i in outflow:
    m2.addConstr(outflow[i] <= travelling_arcs[(str(int(i)-1),i)])
    
#Bus Capacity
for i in arcs_travelling:
    m2.addConstr(bus_cap >= travelling_arcs[i], name = 'Bus Capacity' + str(i))
    
#Diagonal constraints
#for i in arcs_diagonal:
#    m2.addConstr(bus_cap >= diagonal_arcs[i], name = 'Bus Capacity' + str(i))

m2.optimize()
        
print("Primal Solutions")
for v in m2.getVars():
    print("%s %g" % (v.varName, v.x))


Optimize a model with 52 rows, 50 columns and 132 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 37 rows and 21 columns
Presolve time: 0.01s
Presolved: 15 rows, 29 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7294000e+05   2.187500e+01   0.000000e+00      0s
      10    5.7294000e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  5.729400000e+05
Primal Solutions
Arcs Waiting[1,4] 90
Arcs Waiting[2,5] 0
Arcs Waiting[3,6] 0
Arcs Waiting[4,7] 180
Arcs Waiting[5,8] 0
Arcs Waiting[6,9] 0
Arcs Waiting[7,10] 270
Arcs Waiting[8,11] 0
Arcs Waiting[9,12] 0
Arcs Waiting[10,13] 360
Arcs Waiting[11,14] 0
Arcs Waiting[12,15] 0
Arcs Waiting[13,16] 450
Arcs Waiting[14,17] 0
Arcs Waiting[16,19] 540
Arcs Waiting[17,20] 20
Arcs travelling[1,2] 50
Arcs travelling[

In [34]:
# Print objective value
print('Obj: %g' % m2.objVal)

Obj: 572940


In [35]:
# Print shadow prices and sensitivity ranges
m2.printAttr(['Pi','SARHSLow','SARHSUp'])


  Constraint           Pi     SARHSLow      SARHSUp 
---------------------------------------------------
Node 1 Conservation         1049           50       1e+100 
Node 2 Conservation            0          -50          -15 
Node 3 Conservation            0          -50          -15 
Node 4 Conservation        -1039      -1e+100           40 
Node 5 Conservation            0          -50          -15 
Node 6 Conservation            0          -50          -15 
Node 7 Conservation        -1029      -1e+100          130 
Node 8 Conservation            0          -50          -15 
Node 9 Conservation            0          -50          -15 
Node 10 Conservation        -1019      -1e+100          220 
Node 11 Conservation            0          -50          -15 
Node 12 Conservation            0          -50          -15 
Node 13 Conservation        -1009      -1e+100          310 
Node 14 Conservation            0          -50          -15 
Node 15 Conservation            0          -50   